Aluno: Rafael Brunoni

**Motivação:** resolvi buscar dados de saúde por uma questão de interesse pessoal e profissional. Encontrei um a série temporal com estatísticas populacionais de diversos países do mundo.

Dado o vasto número de informações, filtrei dados relacionados ao Brasil e escolhi dois indicadores:
1- % da População com mais de 65 anos: a população brasileira está envelhecendo e essa mudança na pirâmide etária traz consequências em diversos setores da economia como serviços, saúde, previdência, etc...
2- Expctativa de vida da população: apenas como ilustração, inclui um gráfico da expectativa de vida da população brasileira que é um dos fatores que fazem com que a quantidade relativa de idosos cresça no país.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/health-nutrition-and-population-statistics/data.csv')
df.head()

In [ ]:
paises = df['Country Name'].unique()
len(paises)

In [ ]:
indicador = df['Indicator Name'].unique()
len(indicador)

O data set possui 345 indicadores diferentes de 258 países, em um período de tempo de 1960 até 2015.

# Trazendo os dados do Brasil:

In [ ]:
bra = df.loc[df['Country Name'] ==\
'Brazil']
bra

Filtrando o indicador de % da população acima de 65 anos (população idosa):

In [ ]:
pop65 = bra.loc[bra['Indicator Name']==\
'Population ages 65 and above (% of total)']
pop65

Filtrando o indicador de expectativa de vida:

In [ ]:
life = bra.loc[bra['Indicator Name']==\
'Life expectancy at birth, total (years)']
life

Montando o dataframe dos dois indicadores para o Brasil:

In [ ]:
timeseries = pd.concat([pop65, life])
timeseries

In [ ]:
timeseries.shape

In [ ]:
PopAno = pop65.iloc[0,4:60]
pd.to_numeric(PopAno)
PopAno

In [ ]:
##PopAno = pop65.iloc[0,4:60]
##pop = PopAno.to_numpy()
##date = pd.date_range(start='1960', periods=56 , freq = 'Y')
##PopAno = pd.DataFrame({ "pop": teste} , index = date)
##PopAno = pd.to_numeric(PopAno['pop'])

In [ ]:
LifeAno = life.iloc[0,4:60]
pd.to_numeric(LifeAno)
LifeAno

# Usando **Matplotlib**

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
## eixo X = anos eixo y = qauntidade de pessoas acima de 65 anos
plt.figure(figsize=(15, 10)) # alterar tamanho
plt.plot(PopAno)
plt.xlabel('Ano')
plt.ylabel('% da pop total')
plt.title("População anual brasileira > 65 anos ")
plt.rcParams['xtick.labelsize'] = 10
plt.xticks(rotation=90)
plt.show()

Percebe-se no gráfico que o percentual da população brasileira com 65 anos ou mais vem crescendo bastante. Em 1960, um pouco mais de 3% da população estava nessa classificação etária, enquanto em 2015, quase 8% da população brasileira tem mais do que 65 anos. Esse perfil mostra o envelhecimento da população e uma tendência de inversão da pirâmide etária no país.

In [ ]:
plt.figure(figsize=(15, 10)) # alterar tamanho
plt.plot(LifeAno)
plt.xlabel('Ano')
plt.ylabel('Idade')
plt.title("Expectativa de vida da população - Brasil")
plt.rcParams['xtick.labelsize'] = 10
plt.xticks(rotation=90)
plt.show()

> # Usando **Plotly**

In [ ]:
import cufflinks as cf
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px

In [ ]:
PopAno.columns = ['date', 'pop']
pop = PopAno.to_numpy()
date = pd.date_range(start='1960', periods=56 , freq = 'Y')
PopAno = pd.DataFrame({ "pop": pop} , index = date)
PopAno = pd.to_numeric(PopAno['pop'])
PopAno

1- População idosa:

In [ ]:
fig = px.scatter(PopAno,
          x=date,
          y=pop)
fig.show()

2- Expectativa de vida da população:

In [ ]:
idade = LifeAno.to_numpy()
LifeAno.columns = ['date', 'idade']
fig = px.scatter(LifeAno,
          x=date,
          y=idade)
fig.show()

Comportamento das variáveis mostra que o aumento da expectativa de vida da população acaba acarretando no aumento da proporção da população idosa no total da população do país.

# Decompondo a série temporal

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import pyplot

População Idosa:

In [ ]:
result = seasonal_decompose(PopAno, model='multiplicative', period = 1 )
result.plot()
pyplot.show()

Modelo parece ser multiplicativo, uma vez que a observação do seu comportamento ao longo do tempo é de aceleração não linear. Buscando referências na internet sobre o tema, encontra-se muita informação que converge para esta constatação:

*Para o Brasil, de acordo com Camarano (2002), o número de idosos (maiores de 60 anos, seguindo a determinação legal no Brasil) deve quase duplicar em 35 anos, assim como o número de maiores de 70 anos irá triplicar até 2050, chegando a 13,2% da população (IBGE, 2008).*

O modelo não possui sazonalidade.

In [ ]:
print(result.trend)

In [ ]:
print(result.seasonal)

In [ ]:
print(result.resid)

In [ ]:
PopAno.head()

In [ ]:
##PopAno.iloc[,1]

Usando o modelo Autoregression (AR), adequado para séries sem componentes sazonais

In [ ]:
# AR example
from statsmodels.tsa.ar_model import AutoReg
from random import random
from fbprophet import Prophet
# contrived dataset
## PopAno
# fit model
PopAno =  pd.to_numeric(PopAno)
PopAno
model = AutoReg(PopAno, lags=1)
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(len(PopAno), len(PopAno))
##yhat = model_fit.predict(PopAno)
print(yhat)
print(model_fit.summary().tables[1])

In [ ]:
figpredict = px.scatter(yhat,
          x=date,
          y=pop)
figpredict.show()

In [ ]:
model_fit.plot_predict(start=pd.to_datetime('2010-12-31'), end = pd.to_datetime('2020-12-31'), dynamic=False)
plt.show()

In [ ]:
# HWES example
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from random import random
# contrived dataset
##PopAno
# fit model
model = ExponentialSmoothing(PopAno)
model_fit_hwes = model.fit()
# make prediction
yhat2 = model_fit_hwes.predict(len(PopAno), len(PopAno))
print(yhat2)
print(model_fit_hwes.summary().tables[1])

In [ ]:
figpredict = px.scatter(yhat2,
          x=date,
          y=pop)
figpredict.show()